### Задача 1
---
На первом складе (А1) содержится сталь двух марок: 3200 т марки «а» и 5400 т марки «б». Сталь этих марок содержится также и на втором складе (А2) в следующих количествах: 2700 т марки «а» и 5100 т марки «б». Сталь должна быть вывезена в два пункта потребления: в пункт В1 необходимо поставить 1500 т стали марки «а», 6300 т марки «б» и остальные 900 т стали любой марки. Аналогично второй пункт потребления В2 должен получить 5000 т стали, из них 1700 т марки «а» и 3000 т марки «б». Известно, что 1 т стали марки «а» может заменить 0.7 т стали марки «б» (то есть, вместо стали марки «б» можно использовать соответствующее количество стали марки «а», но не наоборот).
Стоимость перевозок в денежных единицах (ДЕ) за тонну составляют: из пункта А1 в пункты В1 и В2 3.4 ДЕ и 4.3 ДЕ, и из пункта А2 в В1 и В2, соответственно, 4.8 ДЕ и 4.3 ДЕ. Требуется найти план перевозок стали, минимизирующий затраты на перевозки; полученное решение необходимо исследовать.
1. Провести параметрическое изменение правых частей:
     - если в решении отсутствуют перевозки с заменой марок стали («а» на «б»), то провести постепенное изменение запасов на обоих складах (увеличение - для стали марки «а», и уменьшение для стали марки «б») до тех пор, пока по крайней мере 100 т стали марки «а» не будут направлены на замену стали марки «б»;
     - если в решении присутствуют такие перевозки, то провести обратное постепенное изменение запасов, пока перевозки с заменой марок стали не исчезнут (рассмотреть вариант только с увеличением запасов стали марки «б» без изменения запасов стали марки «а» на обоих складах).
2. Провести параметрическое изменение целевой функции:
    - предварительно существено увеличить запасы стали обеих марок (в правых частях), чтобы на предпочтительность перевозок стали той или иной марки не влияли ограничения по запасам;
    - убедиться в отсутствии замен (или в их наличии), для чего заново решить задачу;
    - затем провести постепенное изменение (увеличение или уменьшение - по смыслу) стоимости перевозок стали марки «б» (при прежней стоимости перевозок стали марки «а») до появления хотя бы 100 т замены (или до исчезновения замен).
